In [ ]:
from langgraph.graph import Graph, StateGraph, END
from typing import Dict, List, TypedDict, Optional, Annotated
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage
import json
from datetime import datetime
import uuid

# Define the state structure
class ConversationState(TypedDict):
    messages: Annotated[List[Dict], "The conversation messages"]
    user_id: str
    current_intent: str
    loan_type: Optional[str]
    loan_status: Optional[str]
    user_context: Dict
    documents_verified: bool
    application_data: Dict
    tone: str
    needs_human_review: bool
    human_feedback_score: Optional[float]
    conversation_history: List[Dict]

# Initialize components
llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
embeddings = OpenAIEmbeddings()

# Create vector stores for different purposes
cache_memory_store = Chroma(embedding_function=embeddings, collection_name="cache_memory")
conversation_history_store = Chroma(embedding_function=embeddings, collection_name="conversation_history")
loan_documents_store = Chroma(embedding_function=embeddings, collection_name="loan_documents")

# Pre-populate with some loan information
loan_info_docs = [
    Document(page_content="Education loan requirements: Minimum age 18, academic records, co-signer may be required", metadata={"type": "education_loan", "section": "requirements"}),
    Document(page_content="Home loan eligibility: Credit score 650+, stable income, property documents", metadata={"type": "home_loan", "section": "eligibility"}),
    Document(page_content="Education loan interest rates: 6.5% to 11.5% depending on institution and course", metadata={"type": "education_loan", "section": "interest_rates"}),
]
cache_memory_store.add_documents(loan_info_docs)

class LoanManagementSystem:
    def __init__(self):
        self.workflow = self._create_workflow()
    
    def _detect_tone_and_intent(self, state: ConversationState) -> ConversationState:
        """Detect tone and intent of user message"""
        latest_message = state["messages"][-1]["content"]
        
        # Simple tone detection (in production, use more sophisticated NLP)
        tone_keywords = {
            "urgent": ["urgent", "immediately", "asap", "quick", "fast"],
            "frustrated": ["angry", "frustrated", "annoyed", "disappointed"],
            "happy": ["thank", "appreciate", "good", "excellent", "happy"],
            "neutral": ["hi", "hello", "hey", "information"]
        }
        
        detected_tone = "neutral"
        for tone, keywords in tone_keywords.items():
            if any(keyword in latest_message.lower() for keyword in keywords):
                detected_tone = tone
                break
        
        # Intent detection
        intent_keywords = {
            "loan_inquiry": ["education loan", "home loan", "loan information", "interest rate", "eligibility"],
            "loan_status": ["status", "application status", "where is my", "track my"],
            "general_chat": ["hi", "hello", "how are you", "what's up", "good morning"],
            "new_application": ["apply", "want to take", "need a loan", "application"]
        }
        
        detected_intent = "general_chat"
        for intent, keywords in intent_keywords.items():
            if any(keyword in latest_message.lower() for keyword in keywords):
                detected_intent = intent
                break
        
        # Update state
        state["tone"] = detected_tone
        state["current_intent"] = detected_intent
        
        # Extract loan type if mentioned
        if "education" in latest_message.lower():
            state["loan_type"] = "education"
        elif "home" in latest_message.lower():
            state["loan_type"] = "home"
        
        return state

    def _handle_general_conversation(self, state: ConversationState) -> ConversationState:
        """Handle general conversation like greetings"""
        latest_message = state["messages"][-1]["content"].lower()
        
        responses = {
            "greeting": "Hello! I'm your loan assistant. How can I help you with your loan needs today?",
            "how_are_you": "I'm doing great, thank you for asking! I'm here to help you with loan information and applications.",
            "default": "I'm here to assist you with education loans, home loans, checking application status, and general loan information. How can I help you?"
        }
        
        if any(word in latest_message for word in ["hi", "hello", "hey"]):
            response = responses["greeting"]
        elif any(word in latest_message for word in ["how are you", "what's up"]):
            response = responses["how_are_you"]
        else:
            response = responses["default"]
        
        state["messages"].append({"role": "assistant", "content": response})
        return state

    def _handle_loan_information(self, state: ConversationState) -> ConversationState:
        """Handle loan information requests with cache and database lookup"""
        query = state["messages"][-1]["content"]
        loan_type = state.get("loan_type", "general")
        
        # Check cache memory first
        cache_results = cache_memory_store.similarity_search(query, k=2)
        
        if cache_results:
            # Found in cache
            response = f"Based on our records for {loan_type} loans:\n\n"
            for doc in cache_results:
                response += f"• {doc.page_content}\n"
            response += "\nIs there anything specific you'd like to know?"
        else:
            # Simulate database lookup
            db_response = self._query_loan_database(query, loan_type)
            if db_response:
                response = db_response
                # Add to cache for future
                cache_memory_store.add_documents([
                    Document(page_content=db_response, metadata={"type": loan_type, "query": query})
                ])
            else:
                # Fallback to internet search (simulated)
                internet_response = self._search_internet_for_loan_info(query)
                response = internet_response
        
        state["messages"].append({"role": "assistant", "content": response})
        return state

    def _query_loan_database(self, query: str, loan_type: str) -> str:
        """Simulate database query for loan information"""
        # In real implementation, this would query your actual database
        loan_data = {
            "education": {
                "interest_rates": "Education loan interest rates range from 6.5% to 11.5%",
                "eligibility": "Must be enrolled in recognized institution, age 18+, co-signer may be required",
                "documents": "Academic records, income proof, identity proof, admission letter"
            },
            "home": {
                "interest_rates": "Home loan interest rates range from 8.5% to 12%",
                "eligibility": "Credit score 650+, stable income, property documents",
                "documents": "Income proof, property papers, identity proof, address proof"
            }
        }
        
        if loan_type in loan_data:
            return loan_data[loan_type].get("interest_rates", "Information not available in database")
        return ""

    def _search_internet_for_loan_info(self, query: str) -> str:
        """Simulate internet search for loan information"""
        # In real implementation, this would use web search APIs
        return f"I found this information about '{query}': Current market rates and eligibility criteria may vary. For the most accurate information, please visit our official website or contact our loan specialists."

    def _handle_loan_status(self, state: ConversationState) -> ConversationState:
        """Handle loan status inquiries with document verification"""
        user_id = state["user_id"]
        
        # Check if documents are verified
        if not state.get("documents_verified", False):
            # Document verification process
            verification_result = self._verify_documents(user_id)
            state["documents_verified"] = verification_result["verified"]
            
            if not verification_result["verified"]:
                response = "Your documents are currently under verification. This process usually takes 24-48 hours. We'll notify you once completed."
                state["messages"].append({"role": "assistant", "content": response})
                return state
        
        # Get loan status
        loan_status = self._check_loan_status(user_id)
        state["loan_status"] = loan_status["status"]
        
        response = f"Your {state.get('loan_type', 'loan')} application status: {loan_status['status']}\n"
        response += f"Current stage: {loan_status['stage']}\n"
        if loan_status.get('next_steps'):
            response += f"Next steps: {loan_status['next_steps']}"
        
        state["messages"].append({"role": "assistant", "content": response})
        return state

    def _verify_documents(self, user_id: str) -> Dict:
        """Simulate document verification process"""
        # In real implementation, integrate with actual verification system
        return {
            "verified": True,
            "timestamp": datetime.now().isoformat(),
            "verified_by": "auto_system"
        }

    def _check_loan_status(self, user_id: str) -> Dict:
        """Check loan application status"""
        # In real implementation, query your loan management system
        status_options = [
            {"status": "Under Review", "stage": "Document Verification", "next_steps": "Wait for verification completion"},
            {"status": "Approved", "stage": "Sanction Processing", "next_steps": "Sanction letter will be issued within 2 days"},
            {"status": "Documents Requested", "stage": "Additional Documents", "next_steps": "Please submit the requested documents"}
        ]
        
        import random
        return random.choice(status_options)

    def _handle_new_application(self, state: ConversationState) -> ConversationState:
        """Handle new loan application with intent creation"""
        loan_type = state.get("loan_type", "general")
        
        if loan_type == "home":
            return self._process_home_loan_application(state)
        else:
            return self._process_general_loan_application(state)

    def _process_home_loan_application(self, state: ConversationState) -> ConversationState:
        """Process home loan application through multiple agents"""
        
        # Sales Agent
        sales_response = self._sales_agent_interaction(state)
        state["messages"].append({"role": "assistant", "content": sales_response})
        
        # Verification Agent
        verification_result = self._verification_agent(state)
        state["application_data"]["verification"] = verification_result
        
        # Underwriting Agent
        underwriting_result = self._underwriting_agent(state)
        state["application_data"]["underwriting"] = underwriting_result
        
        # Sanction Agent
        sanction_result = self._sanction_agent(state)
        state["application_data"]["sanction"] = sanction_result
        
        final_response = f"Home loan application processed!\n\n"
        final_response += f"Sales Assessment: {sales_response}\n"
        final_response += f"Verification: {verification_result['status']}\n"
        final_response += f"Underwriting: {underwriting_result['decision']}\n"
        final_response += f"Sanction: {sanction_result['status']}\n\n"
        final_response += "Your application is now complete and under final review."
        
        state["messages"].append({"role": "assistant", "content": final_response})
        return state

    def _sales_agent_interaction(self, state: ConversationState) -> str:
        """Sales agent for intent creation and initial assessment"""
        questions = [
            "What is your annual income?",
            "What is the property value you're looking at?",
            "What loan amount are you seeking?",
            "What is your current employment status?"
        ]
        
        # In full implementation, this would be a conversational flow
        return "I've gathered your basic information for the home loan. Let me transfer you to verification."

    def _verification_agent(self, state: ConversationState) -> Dict:
        """Document verification agent"""
        return {
            "status": "Documents verified",
            "timestamp": datetime.now().isoformat(),
            "agent": "verification_system"
        }

    def _underwriting_agent(self, state: ConversationState) -> Dict:
        """Underwriting agent for risk assessment"""
        return {
            "decision": "Approved with standard terms",
            "risk_level": "Medium",
            "loan_to_value": "75%",
            "timestamp": datetime.now().isoformat()
        }

    def _sanction_agent(self, state: ConversationState) -> Dict:
        """Sanction agent for final approval"""
        return {
            "status": "Sanction letter prepared",
            "sanction_amount": "Based on assessment",
            "interest_rate": "9.5%",
            "timestamp": datetime.now().isoformat()
        }

    def _process_general_loan_application(self, state: ConversationState) -> ConversationState:
        """Process general loan applications"""
        response = f"Thank you for your interest in {state.get('loan_type', 'loan')} application. "
        response += "Please provide your basic details and we'll guide you through the process."
        
        state["messages"].append({"role": "assistant", "content": response})
        return state

    def _store_conversation(self, state: ConversationState) -> ConversationState:
        """Store conversation in database"""
        conversation_data = {
            "user_id": state["user_id"],
            "timestamp": datetime.now().isoformat(),
            "messages": state["messages"],
            "intent": state["current_intent"],
            "loan_type": state.get("loan_type"),
            "tone": state["tone"]
        }
        
        # Store in vector database for context retrieval
        conversation_text = " ".join([f"{msg['role']}: {msg['content']}" for msg in state["messages"]])
        conversation_history_store.add_documents([
            Document(
                page_content=conversation_text,
                metadata=conversation_data
            )
        ])
        
        # Update user context
        state["user_context"]["last_interaction"] = datetime.now().isoformat()
        state["user_context"]["conversation_count"] = state["user_context"].get("conversation_count", 0) + 1
        
        return state

    def _human_review_check(self, state: ConversationState) -> str:
        """Check if human review is needed based on tone and content"""
        if state["tone"] in ["frustrated", "urgent"]:
            state["needs_human_review"] = True
            return "human_review"
        
        # Check for complex queries that might need human intervention
        complex_keywords = ["complaint", "escalate", "manager", "speak to human"]
        latest_message = state["messages"][-1]["content"].lower()
        
        if any(keyword in latest_message for keyword in complex_keywords):
            state["needs_human_review"] = True
            return "human_review"
        
        return "continue"

    def _human_review_process(self, state: ConversationState) -> ConversationState:
        """Simulate human review process"""
        # In real implementation, this would connect to human review system
        state["human_feedback_score"] = 1.0  # Assume positive feedback for now
        state["needs_human_review"] = False
        
        state["messages"].append({
            "role": "assistant", 
            "content": "A human agent has reviewed our conversation and confirmed the information provided is accurate."
        })
        
        return state

    def _penalty_reward_system(self, state: ConversationState) -> ConversationState:
        """Implement penalty/reward system based on human feedback"""
        feedback_score = state.get("human_feedback_score")
        
        if feedback_score is not None:
            if feedback_score <= 1.0:  # Poor feedback
                # Log the issue for model improvement
                print(f"Penalty applied for conversation with user {state['user_id']}")
                # In production, this would update model training data
            elif feedback_score >= 1.5:  # Good feedback
                print(f"Positive feedback recorded for user {state['user_id']}")
                # Reward the successful interaction
        
        return state

    def _retrieve_user_context(self, state: ConversationState) -> ConversationState:
        """Retrieve user's previous context when they re-login"""
        user_id = state["user_id"]
        
        # Search for previous conversations
        previous_conversations = conversation_history_store.similarity_search(
            user_id, k=3, filter={"user_id": user_id}
        )
        
        if previous_conversations:
            # Build context from previous interactions
            context = {
                "previous_loans": [],
                "interaction_history": [],
                "preferences": {}
            }
            
            for conv in previous_conversations:
                metadata = conv.metadata
                if metadata.get("loan_type"):
                    context["previous_loans"].append(metadata["loan_type"])
                context["interaction_history"].append({
                    "timestamp": metadata.get("timestamp"),
                    "intent": metadata.get("intent")
                })
            
            state["user_context"] = context
            
            welcome_back = "Welcome back! I can see we've discussed "
            if context["previous_loans"]:
                welcome_back += f"{', '.join(set(context['previous_loans']))} loans before. "
            welcome_back += "How can I help you today?"
            
            state["messages"].append({"role": "assistant", "content": welcome_back})
        
        return state

    def _create_workflow(self) -> Graph:
        """Create the complete workflow graph"""
        workflow = StateGraph(ConversationState)
        
        # Define nodes
        workflow.add_node("detect_tone_intent", self._detect_tone_and_intent)
        workflow.add_node("retrieve_context", self._retrieve_user_context)
        workflow.add_node("general_conversation", self._handle_general_conversation)
        workflow.add_node("loan_information", self._handle_loan_information)
        workflow.add_node("loan_status", self._handle_loan_status)
        workflow.add_node("new_application", self._handle_new_application)
        workflow.add_node("human_review", self._human_review_process)
        workflow.add_node("store_conversation", self._store_conversation)
        workflow.add_node("penalty_reward", self._penalty_reward_system)
        
        # Define entry point
        workflow.set_entry_point("detect_tone_intent")
        
        # Add conditional edges
        workflow.add_conditional_edges(
            "detect_tone_intent",
            self._human_review_check,
            {
                "human_review": "human_review",
                "continue": "retrieve_context"
            }
        )
        
        workflow.add_conditional_edges(
            "retrieve_context",
            lambda state: state["current_intent"],
            {
                "general_chat": "general_conversation",
                "loan_inquiry": "loan_information", 
                "loan_status": "loan_status",
                "new_application": "new_application"
            }
        )
        
        # Connect the main flow
        workflow.add_edge("general_conversation", "store_conversation")
        workflow.add_edge("loan_information", "store_conversation")
        workflow.add_edge("loan_status", "store_conversation")
        workflow.add_edge("new_application", "store_conversation")
        workflow.add_edge("human_review", "store_conversation")
        workflow.add_edge("store_conversation", "penalty_reward")
        workflow.add_edge("penalty_reward", END)
        
        return workflow.compile()

    def process_message(self, user_id: str, message: str, is_new_user: bool = False) -> Dict:
        """Process a user message through the workflow"""
        
        # Initialize state
        initial_state = {
            "messages": [{"role": "user", "content": message}],
            "user_id": user_id,
            "current_intent": "",
            "loan_type": None,
            "loan_status": None,
            "user_context": {} if is_new_user else {"is_returning": True},
            "documents_verified": False,
            "application_data": {},
            "tone": "neutral",
            "needs_human_review": False,
            "human_feedback_score": None,
            "conversation_history": []
        }
        
        # Execute workflow
        final_state = self.workflow.invoke(initial_state)
        
        return {
            "response": final_state["messages"][-1]["content"],
            "user_id": user_id,
            "intent": final_state["current_intent"],
            "loan_type": final_state.get("loan_type"),
            "needs_human_review": final_state["needs_human_review"],
            "tone": final_state["tone"]
        }

# Usage example
if __name__ == "__main__":
    loan_system = LoanManagementSystem()
    
    # Test different scenarios
    test_cases = [
        # First time user - education loan inquiry
        ("user123", "Hi, I want information about education loans", True),
        
        # General conversation
        ("user123", "Hello, how are you doing today?", False),
        
        # Loan status check
        ("user123", "I have applied for education loan, what is my application status?", False),
        
        # New home loan application
        ("user456", "I want to apply for a home loan", True),
    ]
    
    for user_id, message, is_new_user in test_cases:
        print(f"\n{'='*50}")
        print(f"User: {user_id}")
        print(f"Message: {message}")
        print(f"New User: {is_new_user}")
        print(f"{'-'*50}")
        
        result = loan_system.process_message(user_id, message, is_new_user)
        print(f"Response: {result['response']}")
        print(f"Detected Intent: {result['intent']}")
        print(f"Loan Type: {result['loan_type']}")
        print(f"Tone: {result['tone']}")
        print(f"Needs Human Review: {result['needs_human_review']}")